## Binary structure classification used in tree building: Step 3. BiMPM

Prepare data and model-related scripts.

Evaluate models.

Output:
 - ``models/structure_predictor_bimpm/*``

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

### Make a directory

In [ ]:
MODEL_PATH = 'models/structure_predictor_bimpm'

In [ ]:
! mkdir $MODEL_PATH

TRAIN_FILE_PATH = os.path.join(MODEL_PATH, 'structure_cf_train.tsv')
DEV_FILE_PATH = os.path.join(MODEL_PATH, 'structure_cf_dev.tsv')
TEST_FILE_PATH = os.path.join(MODEL_PATH, 'structure_cf_test.tsv')

### Prepare train/test sets

In [ ]:
IN_PATH = 'data_structure'

train_samples = pd.read_pickle(os.path.join(IN_PATH, 'train_samples.pkl'))
dev_samples = pd.read_pickle(os.path.join(IN_PATH, 'dev_samples.pkl'))
test_samples = pd.read_pickle(os.path.join(IN_PATH, 'test_samples.pkl'))

In [ ]:
print(train_samples.shape)
train_samples.relation.value_counts()

In [ ]:
train_samples[train_samples.snippet_x.map(len) < 5].head(1)

In [ ]:
length = train_samples.snippet_x.map(lambda row: len(row.split()))
length.describe()

In [ ]:
train_samples.snippet_y

In [ ]:
train_samples.same_paragraph.value_counts()

### Undersample negative examples 
... or there will not be enough memory in the world to train

here 75259 train samples turn into 41356 samples, x.5

In [ ]:
# We won't discern any sample pairs that are longer than 100 tokens by default,
# and they are bad for the classifier. Filter such pairs:
max_len = 100
train_samples['cutted_snippet_x'] = train_samples.snippet_x.map(lambda row: ' '.join(row.split()[:max_len]))
train_samples['cutted_snippet_y'] = train_samples.snippet_y.map(lambda row: ' '.join(row.split()[:max_len]))
train_samples = train_samples.drop_duplicates(['cutted_snippet_x', 'cutted_snippet_y', 'relation'])
train_samples = train_samples.drop_duplicates(['cutted_snippet_x', 'cutted_snippet_y'], keep=False)
# (75205, 2066)

In [ ]:
# """ Precompute similarities between vectors of negative examples for undersampling.
#     Uncomment and run one time to obtain ``neg_examples_svd.npy`` and/or ``neg_examples_sim.npy``  """

from sklearn.decomposition import TruncatedSVD

# Select negative examples
train_samples_neg = train_samples[train_samples.relation == 0]

# Convert filenames (train set) to numbers, so we won't lose entire documents in undersampling
filenames = pd.get_dummies(train_samples_neg['filename'])
train_samples_neg = pd.concat([train_samples_neg, filenames], axis=1)

# Select digital columns
digital_columns = [column for column in train_samples_neg.columns 
                   if train_samples_neg[column].dtype.name in ['float64', 'int64']]
X_train = train_samples_neg[digital_columns]

# Reduce dimentions
svd = TruncatedSVD(n_components=50)
X_train_reduced = svd.fit_transform(X_train) 
np.save('neg_examples_svd.npy', X_train_reduced.astype(np.float32))

In [ ]:
from sklearn.cluster import MiniBatchKMeans

km = MiniBatchKMeans(n_clusters=train_samples[train_samples.relation == 1].shape[0], 
                     verbose=1, batch_size=8192, max_no_improvement=100)
prediction = km.fit_predict(X_train_reduced.astype(np.float32))

In [ ]:
train_samples_neg['km_cluster'] = prediction

In [ ]:
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

nearest_centroid_dist = []
for idx, km_cluster in tqdm(enumerate(km.labels_), total=km.labels_.shape[0]):
    dist = 1. - cosine_similarity([X_train_reduced[idx]], [km.cluster_centers_[km_cluster]])
    nearest_centroid_dist.append(dist[0][0])
    
train_samples_neg['dist2centroid'] = nearest_centroid_dist

In [ ]:
train_samples_neg.dist2centroid.describe()

In [ ]:
def select_optimal(contents, threshold=None):
    contents = contents.sort_values(by='dist2centroid', ascending=True)
    contents.iloc[-1,-1] = True
    return contents

if 'keep' in train_samples_neg.columns:
    train_samples_neg.drop(columns=['keep'])
train_samples_neg['keep'] = False
train_samples_neg = train_samples_neg.groupby('km_cluster', as_index=False).progress_apply(
    lambda row: select_optimal(row, threshold=train_samples_neg.dist2centroid.describe()['75%']))

In [ ]:
examples = train_samples_neg[['snippet_x', 'snippet_y', 'km_cluster', 'dist2centroid']]

In [ ]:
examples[examples.km_cluster == 1]

In [ ]:
keys = train_samples_neg.keys()
[key for key in keys if 'parag' in key]

In [ ]:
train_samples_neg.same_paragraph.value_counts()

In [ ]:
undersampled = train_samples_neg[train_samples_neg.keep]

In [ ]:
undersampled.shape

In [ ]:
train_samples.relation.value_counts()

In [ ]:
undersampled = undersampled.reset_index(drop=True)

In [ ]:
undersampled.filename.unique().shape[0], train_samples_neg.filename.unique().shape[0]

In [ ]:
del train_samples_neg

In [ ]:
train_samples = pd.concat([train_samples[train_samples.relation == 1], undersampled]).sample(frac=1)

### Save train, dev and test data 

In [ ]:
train_samples.shape

In [ ]:
train_samples[['snippet_x', 'snippet_y', 'relation', 'filename']].sort_values('snippet_x').tail()

In [ ]:
train_samples = train_samples.reset_index()
train_samples[['relation', 'snippet_x', 'snippet_y', 'same_sentence', 'same_paragraph', 'index']].to_csv(
    TRAIN_FILE_PATH, sep='\t', header=False, index=False)

dev_samples = dev_samples.reset_index()
dev_samples[['relation', 'snippet_x', 'snippet_y', 'same_sentence', 'same_paragraph', 'index']].to_csv(
    DEV_FILE_PATH, sep='\t', header=False, index=False)

test_samples = test_samples.reset_index()
test_samples[['relation', 'snippet_x', 'snippet_y', 'same_sentence', 'same_paragraph', 'index']].to_csv(
    TEST_FILE_PATH, sep='\t', header=False, index=False)

In [ ]:
TRAIN_FILE_PATH

In [ ]:
! wc -l models/structure_predictor_bimpm/structure_cf_train.tsv

In [ ]:
counts = train_samples['relation'].value_counts(normalize=False).values
NUMBER_CLASSES = len(counts)
print("number of classes:", NUMBER_CLASSES)
print("class weights:", np.round(counts.min() / counts, decimals=6))

###  Generate config file

In [ ]:
print(TRAIN_FILE_PATH)
print(DEV_FILE_PATH)
print(TEST_FILE_PATH)

In [ ]:
MODEL_PATH

In [ ]:
%%writefile $MODEL_PATH/config_elmo_fasttext.jsonnet

// Configuration for a sentence matching model based on:
//   Wang, Zhiguo, Wael Hamza, and Radu Florian. 
//   "Bilateral multi-perspective matching for natural language sentences."
//   Proceedings of the 26th International Joint Conference on Artificial Intelligence. 2017.
// (Augmented with additional granularity related features)


local NUM_EPOCHS = 50;
local LR = std.parseJson(std.extVar('LR'));
local MAX_LEN = 100;
local LSTM_ENCODER_HIDDEN = std.parseJson(std.extVar('LSTM_ENCODER_HIDDEN'));
local LSTM_AGG_HIDDEN = std.parseJson(std.extVar('LSTM_AGG_HIDDEN'));

local dataset_reader_type = "bimpm_custom_package.dataset_readers.custom_reader.CustomDataReader";
local model_type = "bimpm_custom_package.model.custom_bimpm.BiMpm";

{
  "dataset_reader": {
    "type": dataset_reader_type,
    "tokenizer": {
      "type": "just_spaces"
    },
    "token_indexers": {
      "token_characters": {
        "type": "characters",
        "min_padding_length": 30
      },
      "elmo": {
        "type": "elmo_characters",
      },
#       "tokens": {
#         "type": "single_id",
#         "lowercase_tokens": true
#       },
    }
  },
  "train_data_path": "structure_predictor_bimpm/structure_cf_train.tsv",
  "validation_data_path": "structure_predictor_bimpm/structure_cf_dev.tsv",
  "model": {
    "type": model_type,
    "dropout": 0.5,
    "class_weights": [1.0, 1.0],
    "encode_together": false,
    "text_field_embedder": {
      "token_embedders": {
#         "tokens": {
#           "type": "embedding",
#           "embedding_dim": 300,
#           "pretrained_file": "ft_native_300_ru_wiki_lenta_nltk_wordpunct_tokenize.vec",
#           "trainable": false
#         },
        "elmo": {
          "type": "elmo_token_embedder",
          "options_file": "rsv_elmo/options.json",
          "weight_file": "rsv_elmo/model.hdf5",
          "do_layer_norm": false,
          "projection_dim": 100,
          "dropout": 0.0
        },
        "token_characters": {
          "type": "character_encoding",
            "dropout": 0.2,
            "embedding": {
              "embedding_dim": 20,
              "sparse": false,
              "vocab_namespace": "token_characters"
          },
          "encoder": {
            "type": "gru",
            "input_size": $.model.text_field_embedder.token_embedders.token_characters.embedding.embedding_dim,
            "hidden_size": LSTM_ENCODER_HIDDEN,
            "num_layers": 1,
            "bidirectional": true,
          },
        }
      }
    },
    "matcher_word": {
      "is_forward": true,
      "hidden_dim": $.model.text_field_embedder.token_embedders.elmo.projection_dim+LSTM_ENCODER_HIDDEN*2,
      "num_perspectives": 10,
      "with_full_match": false
    },
    "encoder1": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": $.model.text_field_embedder.token_embedders.elmo.projection_dim+LSTM_ENCODER_HIDDEN*2,
      "hidden_size": 50,
      "num_layers": 1
    },
    "matcher_forward1": {
      "is_forward": true,
      "hidden_dim": 50,
      "num_perspectives": 10
    },
    "matcher_backward1": {
      "is_forward": false,
      "hidden_dim": 50,
      "num_perspectives": 10
    },
    "encoder2": {
      "type": "lstm",
      "bidirectional": true,
      "input_size": $.model.matcher_forward1.hidden_dim*2,
      "hidden_size": 50,
      "num_layers": 1
    },
    "matcher_forward2": {
      "is_forward": true,
      "hidden_dim": 50,
      "num_perspectives": 10
    },
    "matcher_backward2": {
      "is_forward": false,
      "hidden_dim": 50,
      "num_perspectives": 10
    },
    "aggregator":{
      "type": "lstm",
      "bidirectional": true,
      "input_size": 264,
      "hidden_size": LSTM_AGG_HIDDEN,
      "num_layers": 1,
      "dropout": 0.1,
    },
    "classifier_feedforward": {
      "input_dim": LSTM_AGG_HIDDEN*4+1+1,
      "num_layers": 1,
      "hidden_dims": [2],
      "activations": ["mish"],
      "dropout": [0.0]
    },
    "initializer": {
      "regexes": [
        [".*linear_layers.*weight", {"type": "xavier_normal"}],
        [".*linear_layers.*bias", {"type": "constant", "val": 0}],
        [".*weight_ih.*", {"type": "xavier_normal"}],
        [".*weight_hh.*", {"type": "orthogonal"}],
        [".*bias.*", {"type": "constant", "val": 0}],
        [".*matcher.*match_weights.*", {"type": "kaiming_normal"}]
      ]
    }
  },
  "data_loader": {
    "type": 'multiprocess',
    "max_instances_in_memory": $.data_loader.batch_sampler.batch_size * 10,
    "batch_sampler": {
      "type": "bucket",
      "batch_size": 2,
      "padding_noise": 0.0,
      "sorting_keys": ["premise"],
    },
  },
  "trainer": {
    "num_epochs": NUM_EPOCHS,
    "patience": 2,
    "grad_clipping": 5.0,
    "validation_metric": "+f1",
    "cuda_device": 1,
    "optimizer": {
      "type": "huggingface_adamw",
      "lr": LR
    },
  }
}

In [ ]:
%%writefile models/structure_predictor_params.json

[
  {
    "type": "int",
    "attributes": {
      "name": "LSTM_ENCODER_HIDDEN",
      "low": 10,
      "high": 20
    }
  },
  {
    "type": "int",
    "attributes": {
      "name": "LSTM_AGG_HIDDEN",
      "low": 25,
      "high": 50
    }
  },
  {
    "type": "float",
    "attributes": {
      "name": "LR",
      "low": 2e-4,
      "high": 2e-2,
      "log": true
    }
  }
]

Optuna

In [ ]:
# ! echo 'allennlp_optuna' >> models/.allennlp_plugins

In [ ]:
%%writefile models/tune_structure_predictor.sh

export METHOD=structure_predictor_bimpm
export STUDY_NAME=structure_tuning_0
mkdir optuna
rm -r optuna/$METHOD
mkdir optuna/$METHOD

allennlp tune ${METHOD}/config_elmo_fasttext.jsonnet structure_predictor_params.json --serialization-dir optuna/$METHOD \
    --study-name $STUDY_NAME \
    --skip-if-exists \
    --metrics best_validation_f1 \
    --direction maximize

In [ ]:
import json

def collect_optuna_results(path):
    for trial in glob.glob(os.path.join(path, 'trial_*/')):
        try:
            metrics = json.load(open(os.path.join(trial, 'metrics.json')))
            print(trial, metrics['best_validation_f1'])
        except:
            pass
        
collect_optuna_results('models/optuna/structure_predictor_bimpm')

In [ ]:
! mv models/optuna/structure_predictor_bimpm/trial_15 models/structure_predictor_bimpm/elmo_ft

In [ ]:
! rm -r models/optuna/structure_predictor_bimpm

### 3. Scripts for training/prediction 

#### Option 1. Directly from the config

Train a model

In [ ]:
%%writefile models/train_structure_predictor.sh
# usage:
# $ cd models 
# $ sh train_structure_predictor.sh {bert|elmo} result_directory

export METHOD=${1}
export RESULT_DIR=${2}
export DEV_FILE_PATH="structure_cf_dev.tsv"
export TEST_FILE_PATH="structure_cf_test.tsv"

rm -r structure_predictor_bimpm/${RESULT_DIR}/
allennlp train -s structure_predictor_bimpm/${RESULT_DIR}/ structure_predictor_bimpm/config_${METHOD}.json \
   --include-package bimpm_custom_package

Predict on dev&test

In [ ]:
%%writefile models/eval_structure_predictor.sh
# usage:
# $ cd models 
# $ sh eval_structure_predictor.sh {bert|elmo}

export METHOD=${1}
export DEV_FILE_PATH="structure_cf_dev.tsv"
export TEST_FILE_PATH="structure_cf_test.tsv"


allennlp predict --use-dataset-reader --cuda-device 0 --silent \
                 --output-file structure_predictor_bimpm/predictions_dev.json structure_predictor_bimpm/${METHOD}/model.tar.gz structure_predictor_bimpm/${DEV_FILE_PATH} \
                 --include-package bimpm_custom_package \
                 --predictor bimpm_custom_package.model.custom_bimpm_predictor.CustomBiMPMPredictor

allennlp predict --use-dataset-reader --cuda-device 0 --silent \
                 --output-file structure_predictor_bimpm/predictions_test.json structure_predictor_bimpm/${METHOD}/model.tar.gz structure_predictor_bimpm/${TEST_FILE_PATH} \
                 --include-package bimpm_custom_package \
                 --predictor bimpm_custom_package.model.custom_bimpm_predictor.CustomBiMPMPredictor

###  Evaluate classifier

In [ ]:
def load_predictions(path, threshold=0.5):
    result = []
    
    with open(path, 'r') as file:
        for line in file.readlines():
            if threshold == 0.5:
                result.append(json.loads(line)["label"])
            else:
                result.append(int(json.loads(line)["probs"][1] > threshold))
            
    result = list(map(int, result))
    return result

In [ ]:
RESULT_DIR = ''

On dev set

In [ ]:
MODEL_PATH = '../../models/structure_predictor_bimpm/'

In [ ]:
! ls $MODEL_PATH

In [ ]:
import pandas as pd

true = pd.read_csv(f'{MODEL_PATH}/structure_cf_dev.tsv', sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_dev.json', threshold=0.6)
print('length of true labels:', len(true))
print('length of prediction:', len(pred))

In [ ]:
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score

print('f1: %.2f'%(f1_score(true[:len(pred)], pred)*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred)*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred)*100))

print(classification_report(true[:len(pred)], pred, digits=4))

On test set

In [ ]:
import pandas as pd
import json

true = pd.read_csv(f'{MODEL_PATH}/structure_cf_test.tsv', sep='\t', header=None)[0].values.tolist()
pred = load_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_test.json')
print('length of true labels:', len(true))
print('length of prediction:', len(pred))

In [ ]:
from sklearn.metrics import classification_report

print('f1: %.2f'%(f1_score(true[:len(pred)], pred)*100))
print('pr: %.2f'%(precision_score(true[:len(pred)], pred)*100))
print('re: %.2f'%(recall_score(true[:len(pred)], pred)*100))

print(classification_report(true[:len(pred)], pred, digits=4))

### Ensemble 

In [ ]:
# %cd ../isanlp_rst
# from classifier_wrappers import *
# %cd ../maintenance/

In [ ]:
model_vocab = [0, 1]
catboost_vocab = [0, 1]

def load_neural_predictions(path):
    result = []
    
    with open(path, 'r') as file:
        for line in file.readlines():
            probs = json.loads(line)['probs']
            probs = {model_vocab[i]: probs[i] for i in range(len(model_vocab))}
            result.append(probs)
            
    return result

def load_scikit_predictions(model, X):
    result = []
    
    try:
        predictions = model.predict_proba(X)
    except AttributeError:
        predictions = model._predict_proba_lr(X)
    
    for prediction in predictions:
        probs = {catboost_vocab[j]: prediction[j] for j in range(len(catboost_vocab))}
        result.append(probs)
    
    return result

def vote_predictions(pred1, pred2, soft=True, weights=[1., 1.]):
    assert len(pred1) == len(pred2)
    result = []
    
    for i in range(len(pred1)):
        sample_result = {}
        for key in pred1[i].keys():
            if soft:
                sample_result[key] = (pred1[i][key]*weights[0] + pred2[i][key]*weights[1]) / 2.
            else:
                sample_result[key] = max(pred1[i][key], pred2[i][key])
        
        result.append(sample_result)
    
    return result

def probs_to_classes(pred):
    result = []
    
    for sample in pred:
        best_class = ''
        best_prob = 0.
        for key in sample.keys():
            if sample[key] > best_prob:
                best_prob = sample[key]
                best_class = key
        
        result.append(best_class)
    
    return result

In [ ]:
import pickle

model = pickle.load(open('../../models/structure_predictor_baseline/model.pkl', 'rb'))
scaler = pickle.load(open('../../models/structure_predictor_baseline/scaler.pkl', 'rb'))
drop_columns = pickle.load(open('../../models/structure_predictor_baseline/drop_columns.pkl', 'rb'))

In [ ]:
IN_PATH = 'data_structure'

train_samples = pd.read_pickle(os.path.join(IN_PATH, 'train_samples.pkl'))
dev_samples = pd.read_pickle(os.path.join(IN_PATH, 'dev_samples.pkl'))
test_samples = pd.read_pickle(os.path.join(IN_PATH, 'test_samples.pkl'))

y_train, X_train = train_samples['relation'].to_frame(), train_samples.drop('relation', axis=1).drop(
    columns=drop_columns + ['category_id'])
y_dev, X_dev = dev_samples['relation'].to_frame(), dev_samples.drop('relation', axis=1).drop(
    columns=drop_columns + ['category_id'])
y_test, X_test = test_samples['relation'].to_frame(), test_samples.drop('relation', axis=1).drop(
    columns=drop_columns + ['category_id'])

In [ ]:
X_scaled_np = scaler.transform(X_dev)
X_dev = pd.DataFrame(X_scaled_np, index=X_dev.index)

X_scaled_np = scaler.transform(X_test)
X_test = pd.DataFrame(X_scaled_np, index=X_test.index)

In [ ]:
model.labels

In [ ]:
from sklearn import metrics


TARGET = 'relation'
svm_predictions = load_scikit_predictions(model, X_dev)
neural_predictions = load_neural_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_dev.json')

tmp = vote_predictions(neural_predictions, svm_predictions, weights=[0.9, 0.8])
ensemble_pred = probs_to_classes(tmp)

print('f1: %.2f'%(metrics.f1_score(y_dev, ensemble_pred)*100.))
print('pr: %.2f'%(metrics.precision_score(y_dev, ensemble_pred)*100.))
print('re: %.2f'%(metrics.recall_score(y_dev, ensemble_pred)*100.))
print()
print(metrics.classification_report(y_dev, ensemble_pred, digits=4))

In [ ]:
pred1 = [_[0] for _ in tmp]

In [ ]:
min(pred1)

In [ ]:
len(neural_predictions) == len(svm_predictions)

#### Statistics of greedy span prediction
use it for document-level threshold adjustment

In [ ]:
tmp_only_0 = [_ for _ in tmp if _[1] <= _[0]]
pred1 = [_[1] for _ in tmp_only_0]
print(pd.Series(pred1).describe())
pd.Series(pred1).plot(kind='density')

In [ ]:
tmp_only_1 = [_ for _ in tmp if _[1] > _[0]]
pred1 = [_[1] for _ in tmp_only_1]
print(pd.Series(pred1).describe())
pd.Series(pred1).plot(kind='density')

In [ ]:
dev_samples[(np.array(ensemble_pred) == 1) & dev_samples.category_id.isna()].sample()[['snippet_x', 'snippet_y']].values

In [ ]:
top_f1 = .5

for w0 in np.arange(0.001, 1., 0.1):
    for w1 in np.arange(0.001, 1., 0.1):
        tmp = vote_predictions(neural_predictions, svm_predictions, soft=True, weights=[w0, w1])
        ensemble_pred = probs_to_classes(tmp)
        f1 = metrics.f1_score(y_dev.values, ensemble_pred, average='macro')
        if f1 > top_f1:
            print(np.round(w0, 2), np.round(w1, 2), '---------------------------------------------------')
            print('f1: %.2f'%(metrics.f1_score(y_dev, ensemble_pred)*100.))
            print('pr: %.2f'%(metrics.precision_score(y_dev, ensemble_pred)*100.))
            print('re: %.2f'%(metrics.recall_score(y_dev, ensemble_pred)*100.))
            print()
            print(metrics.classification_report(y_dev, ensemble_pred, digits=4))
            top_f1 = f1

In [ ]:
svm_predictions = load_scikit_predictions(model, X_test)
neural_predictions = load_neural_predictions(f'{MODEL_PATH}/{RESULT_DIR}/predictions_test.json')

tmp = vote_predictions(neural_predictions, svm_predictions, weights=[0.9, 0.8])
ensemble_pred = probs_to_classes(tmp)

print('f1: %.2f'%(metrics.f1_score(y_test, ensemble_pred)*100.))
print('pr: %.2f'%(metrics.precision_score(y_test, ensemble_pred)*100.))
print('re: %.2f'%(metrics.recall_score(y_test, ensemble_pred)*100.))
print()
print(metrics.classification_report(y_test, ensemble_pred, digits=4))

In [ ]:
tmp_only_0 = [_ for _ in tmp if _[1] <= _[0]]
pred1 = [_[1] for _ in tmp_only_0]
print(pd.Series(pred1).describe())
pd.Series(pred1).plot(kind='density')

In [ ]:
tmp_only_1 = [_ for _ in tmp if _[1] > _[0]]
pred1 = [_[1] for _ in tmp_only_1]
print(pd.Series(pred1).describe())
pd.Series(pred1).plot(kind='density')

### Just for testing 

In [ ]:
! cp -r ../../../_isanlp_rst/rsv_elmo ../../models/rsv_elmo

In [ ]:
! ln -s -r ../../models/rsv_elmo rsv_elmo

In [ ]:
! ls -laht ../../models

In [ ]:
! rm -r ../../models/rvs_elmo

In [ ]:
from allennlp.predictors.predictor import Predictor

clf = Predictor.from_path('../../models/structure_predictor_bimpm/elmo_ft/model.tar.gz', cuda_device=-1, 
                          predictor_name='models.bimpm_custom_package.model.custom_bimpm_predictor.CustomBiMPMPredictor')

In [ ]:
clf.predict(premise='Сейчас я думаю ,', hypothesis='что это в моих интересах ,', same_sentence='1', same_paragraph='1')

In [ ]:
clf.predict(premise='что это в моих интересах ,', hypothesis='потому что ну сколько можно .', same_sentence='1', same_paragraph='1')